In [1]:
import requests
import shutil
import os
import io
import json
import time
import datetime
import dateutil
import random
import re
import ast
import pickle
import numpy as np
import pandas as pd
from IPython.display import clear_output
from pprint import pprint
import bs4
import matplotlib.pyplot as plt
import copy

parent_folder='steam_data'

csv_folder=os.path.join(parent_folder,"bulk","csv_no_text","early_access_standardized")
csv_english_folder=os.path.join(parent_folder,"bulk","csv_english","early_access_standardized")
testing_folder=os.path.join(parent_folder,"bulk","testing")

In [94]:
with open(os.path.join(testing_folder,'still_in_early_access_FINAL'),'r') as file:
    current_ea_games=sorted(list(set([int(x) for x in file.read().split('\n')[:-1]])))
with open(os.path.join(testing_folder,'out_of_early_access_FINAL'),'r') as file:
    former_ea_games=sorted(list(set([int(x) for x in file.read().split('\n')[:-1]])))
full_games_list=sorted(list(set(current_ea_games+former_ea_games)))
with open(os.path.join(parent_folder,'official_appid_json_list'),'r') as file:
    temp_json=json.load(file)['applist']['apps']
official_app_list={}
for x in temp_json:
    official_app_list[x['appid']]=x['name']
print(len(full_games_list))
game_df=pd.read_csv(os.path.join(testing_folder,'20190124_games.csv'))
all_categories=set()
all_genres=set()
all_developers={}
all_publishers={}
for x in ['genres','developers','publishers','categories']:
    game_df[x]=game_df[x].apply(ast.literal_eval)
for i,row in game_df.iterrows():
    all_categories=all_categories | set(row['categories'])
    all_genres=all_genres | set(row['genres'])
    #print(all_genres)
    for developer in row['developers']:
        if developer not in all_developers:
            all_developers[developer]=1
        else:
            all_developers[developer]+=1
    for publisher in row['publishers']:
        if publisher not in all_publishers:
            all_publishers[publisher]=1
        else:
            all_publishers[publisher]+=1
#print(all_genres)
all_genres.remove('Early Access')
all_categories=sorted(list(all_categories))
all_genres=sorted(list(all_genres))
game_df=game_df.set_index('appid')

4935


In [2]:
num_reviews_df=pd.read_csv(os.path.join(testing_folder,'20190126_num_reviews.csv')).set_index('appid')
early_access_start_time_df=pd.read_csv(os.path.join(testing_folder,'20190126_early_access_start_time.csv')).set_index('appid')
normal_review_start_time_df=pd.read_csv(os.path.join(testing_folder,'20190126_normal_review_start_time.csv')).set_index('appid')
num_english_reviews={}

In [11]:
html_out_folder=os.path.join(parent_folder,"bulk","steamspy_html")
if not os.path.exists(html_out_folder):
    os.makedirs(html_out_folder)

for i, appid in enumerate(num_reviews_df.index.tolist(),1):
    try:
        page=requests.get('https://steamspy.com/app/'+str(appid)).text
        foundpage=True
    except:
        foundpage=False
    if foundpage:
        try:
            with open(os.path.join(html_out_folder,str(appid)),'w') as file:
                file.write(page)
        except:
            foundpage=False
    if not foundpage:
        print("ERROR: "+str(i)+": "+str(appid))
    time.sleep(5)
    clear_output(wait=True)
    print(str(i)+"/"+str(len(num_reviews_df)))
print("Done!")

4869/4869


In [43]:
html_folder=os.path.join(parent_folder,"bulk","steamspy_html")

all_tags={}

for filename in os.listdir(html_folder):
    appid=int(filename)
    with open(os.path.join(html_folder,filename),'r') as file:
        html=file.read()
    tag_search_string="<strong>Tags:</strong>"
    tag_start_idx=html.find(tag_search_string)+len(tag_search_string)
    tag_end_idx=html.find("<strong>",tag_start_idx)
    html=html[tag_start_idx:tag_end_idx]
    done=False
    start_idx=-1
    tag_list=[]
    tag_number_list=[]
    while not done:
        start_idx=html.find("<a href=/tag/",start_idx+1)
        if start_idx==-1:
            done=True
            break
        start_idx=html.find(">",start_idx)
        end_idx=html.find("<",start_idx)
        tag_name=html[start_idx+1:end_idx]
        tag_list.append(tag_name)
        if tag_name not in all_tags:
            all_tags[tag_name]=0
        all_tags[tag_name]+=1
        start_idx=end_idx+5
        end_idx=html.find(")",start_idx)
        tag_number_list.append(int(html[start_idx+1:end_idx]))
        if len(tag_list)>5:
            done=True
            break
for tag in list(all_tags.keys()):
    if all_tags[tag]<10:
        all_tags.pop(tag)
    #print(html)
    #break

print(len(all_tags))
print(list(all_tags.keys()))
print(all_categories)
print(all_genres)

168
['Early Access', 'Free to Play', 'Survival', 'Multiplayer', 'Open World', 'Massively Multiplayer', 'Adventure', 'Action', 'RPG', 'Indie', 'Dungeon Crawler', 'Casual', 'Violent', 'Gore', 'Crafting', 'Base-Building', 'Strategy', 'Simulation', 'Sandbox', 'Top-Down Shooter', 'Twin Stick Shooter', 'Dinosaurs', 'Horror', 'Survival Horror', 'Sports', 'Robots', 'Building', 'Visual Novel', 'Anime', 'Sci-fi', 'Economy', 'Singleplayer', 'Parkour', 'First-Person', 'Level Editor', 'VR', 'Funny', 'Tanks', 'Space', 'Shooter', 'Board Game', 'Memes', 'Fighting', '2D Fighter', 'Pixel Graphics', 'Exploration', 'Colorful', 'Metroidvania', 'Racing', 'Card Game', 'Trading Card Game', 'Turn-Based Strategy', 'Animation & Modeling', 'Video Production', 'Comedy', 'Post-apocalyptic', 'PvP', 'Atmospheric', 'Co-op', 'FPS', 'Flight', 'Arcade', 'Utilities', 'Design & Illustration', 'Education', 'Nudity', 'Sexual Content', 'Zombies', 'Clicker', 'Tower Defense', 'Turn-Based Tactics', 'Mature', 'Mechs', 'RTS', 'Onl

In [130]:
import pickle

with open(os.path.join(parent_folder,'bulk','feature_vecs','latest_appids.pkl'),'rb') as file:
    applist_from_words=pickle.load(file)
    
#print(applist_from_words)

In [211]:
out_folder=os.path.join(parent_folder,"bulk","feature_vecs","nonnormalized")
if not os.path.exists(out_folder):
    os.makedirs(out_folder)

tag_sort_dict={'Top-Down Shooter':['Top-Down','Shooter'],
               'Twin Stick Shooter':['Shooter'],
               'FPS':['First-Person','Shooter'],
               'Third-Person Shooter':['Third Person','Shooter'],
               'Arena Shooter':['Shooter'],
               'Psychological Horror':['Horror'],
               'City Builder':['Building','Simulation'],
               'Fishing':['Sports'],
               'Action RPG':['Action','RPG'],
               'Grand Strategy':['4X'],
               'Soccer':['Sports'],
               'Golf':['Sports'],
               'Mini Golf':['Sports'],
               'Local Co-op':['Co-op'],
               'Online Co-Op':['Co-op'],
               'Online Co-op':['Co-op'],
               'RTS':['Strategy','Real-Time'],
               'Turn-Based Strategy':['Turn-Based','Strategy'],
               'Dark Fantasy':['Fantasy'],
               'Pinball':['Arcade'],
               'MMORPG':['Massively Multiplayer','RPG'],
               'Base-Building':['Building'],
               '3D Platformer':['Platformer'],
               '2D Fighter':['2D','Fighting'],
               'Puzzle-Platformer':['Puzzle','Platformer'],
               'Action-Adventure':['Action','Adventure'],
               'Turn-Based Combat':['Turn-Based'],
               'Turn-Based Tactics':['Turn-Based','Tactics'],
               'Comedy':['Funny'],
               'Mechs':['Robots'],
               'Pirates':['Navel'],
               'Great Soundtrack':['Music'],
               'Bullet Hell':["Shoot 'Em Up"],
               'World War II':['War'],
               'Online Multi-Player':['Multiplayer'],
               'Multi-player':['Multiplayer'],
               'Local Multi-Player':['Multiplayer'],
               'MMORPG':['Massively Multiplayer'],
               'Includes level editor':['Level Editor'],
               'Partial Controller Support':['Controller'],
               'Full controller support':['Controller'],
               'Shared/Split Screen':['Split Screen'],
               'VR Support':['VR'],
               'Accounting':['Software'],
               'Animation & Modeling':['Software'],
               'Audio Production':['Software'],
               'Design & Illustration':['Software'],
               'Game Development':['Software'],
               'Photo Editing':['Software'],
               'Software Training':['Software'],
               'Utilities':['Software'],
               'Video Production':['Software'],
               'Web Publishing':['Software']
              }

final_genres_list=['2D','4X','Survival','Open World','Adventure','Action','RPG','Indie','Dungeon Crawler','Casual','Strategy',
        'Simulation','Sandbox','Shooter','Horror','Sports','Visual Novel','First-Person',
        'Top-Down','Third Person','God Game','Rhythm',"Beat 'em up",'Survival Horror','Battle Royale',
        'Puzzle','PvP','Rogue-like','Tactics','Turn-Based','Real-Time','MOBA','Building','VR Only',
        'Platformer','Walking Simulator','Stealth','Side Scroller','Education','Metroidvania','Space Sim',
        'Management','Fighting','Hack and Slash','Flight','Exploration','Isometric','Runner',
        "Shoot 'Em Up",'Arcade','Racing','Board Game','Card Game','Software']
final_themes_list=['Western','Sci-fi','Anime','Realistic','Cyberpunk', 'Historical','Cute', 'Lovecraftian',
                   'Space','Relaxing', 'Magic','Retro','Atmospheric','Fantasy','Science','Steampunk',
                   'Zombies','Futuristic','Minimalist','Funny', 'Post-apocalyptic','Dinosaurs','Medieval',
                   'Tactical','Robots','Military','Naval','Colorful','War','Music']
final_features_list=['Pixel Graphics','Level Editor','Physics', 'Team-Based','Co-op','Multiplayer',
          'Massively Multiplayer','VR','Difficult','Procedural Generation',
          'Character Customization','Controller','Resource Management','Competitive',
          'Rogue-lite','Story Rich','Fast-Paced','Crafting','Split Screen',
                     'Downloadable Content', 'In-App Purchases']
final_genres_list=sorted(final_genres_list)
final_themes_list=sorted(final_themes_list)
final_features_list=sorted(final_features_list)
mature_list=['Nudity', 'Sexual Content','Mature']

def add_feature_to_vector(v,score,mature_tag,feature,discarded_tags,maximize=False):
    f_index=-1
    if feature in mature_list:
        mature_tag[0]=True
    elif feature not in tag_sort_dict:
        try:
            f_index=final_genres_list.index(feature)
            if not maximize:
                v[f_index]=max(v[f_index],score)
            else:
                v[f_index]=np.max(v)
        except:
            pass
        try:
            f_index=final_themes_list.index(feature)
            if not maximize:
                v[len(final_genres_list)+f_index]=max(v[len(final_genres_list)+f_index],score)
            else:
                v[len(final_genres_list)+f_index]=np.max(v)
        except:
            pass
        try:
            f_index=final_features_list.index(feature)
            if not maximize:
                v[len(final_genres_list)+len(final_themes_list)+f_index]=max(v[len(final_genres_list)+len(final_themes_list)+f_index],score)
            else:
                v[len(final_genres_list)+len(final_themes_list)+f_index]=np.max(v)
        except:
            pass
        if f_index==-1:
            if feature.find('RPG')!=-1:
                add_feature_to_vector(v,score,mature_tag,'RPG',discarded_tags,maximize)
            else:
                if feature not in discarded_tags:
                    discarded_tags[feature]=0
                discarded_tags[feature]+=1
    else:
        for key in tag_sort_dict[feature]:
            add_feature_to_vector(v,score,mature_tag,key,discarded_tags,maximize)
            
        
        
appwarnings={'appid':[],
            'content_warning':[],
             'still_in_early_access':[],
             'name':[]
            }
        
discarded_tags={}
v_list=[]
#k=0
dirlist=[int(x) for x in os.listdir(html_folder)]
for i,appid in enumerate(sorted(dirlist),1):
    if appid in applist_from_words:
        filename=str(appid)
        v=np.zeros(len(final_genres_list)+len(final_themes_list)+len(final_features_list))
        mature_tag=[False]
        with open(os.path.join(html_folder,filename),'r') as file:
            html=file.read()
        tag_search_string="<strong>Tags:</strong>"
        tag_start_idx=html.find(tag_search_string)+len(tag_search_string)
        tag_end_idx=html.find("<strong>",tag_start_idx)
        html=html[tag_start_idx:tag_end_idx]
        done=False
        start_idx=-1
        tag_names=[]
        while not done:
            start_idx=html.find("<a href=/tag/",start_idx+1)
            if start_idx==-1:
                done=True
                break
            start_idx=html.find(">",start_idx)
            end_idx=html.find("<",start_idx)
            tag_name=html[start_idx+1:end_idx]
            if tag_name not in all_tags:
                all_tags[tag_name]=0
            all_tags[tag_name]+=1
            start_idx=end_idx+5
            end_idx=html.find(")",start_idx)
            add_feature_to_vector(v,int(html[start_idx+1:end_idx]),mature_tag,tag_name,discarded_tags)

        for tag_name in game_df.loc[appid]['categories']:
            add_feature_to_vector(v,0,mature_tag,tag_name,discarded_tags,maximize=True)
            if tag_name in discarded_tags:
                tag_names.append(tag_name)
        for tag_name in game_df.loc[appid]['genres']:
            add_feature_to_vector(v,0,mature_tag,tag_name,discarded_tags,maximize=True)
            if tag_name in discarded_tags:
                tag_names.append(tag_name)
        if game_df.loc[appid]['content_warning']:
            mature_tag[0]=True
        #k+=1
        #if k<5:
        #    print(v/np.linalg.norm(v))
        if np.linalg.norm(v)!=0:
            appwarnings['appid'].append(appid)
            appwarnings['content_warning'].append(mature_tag[0])
            appwarnings['still_in_early_access'].append(game_df.loc[appid]['still_in_early_access'])
            appwarnings['name'].append(game_df.loc[appid]['name'])
            v_list.append(v)
            np.savetxt(os.path.join(out_folder,str(appid)), v, delimiter=",",fmt='%i')
    clear_output(wait=True)
    print(str(i)+"/"+str(len(dirlist)))

print("Done!")

4869/4869
Done!


In [212]:
appwarnings_df=pd.DataFrame.from_dict(appwarnings)
appwarnings_df.to_csv(os.path.join(parent_folder,"bulk","feature_vecs",'appwarnings.csv'),index=False)

In [213]:
all_vs=np.vstack(v_list)
#print(all_vs[0])
v_df=pd.DataFrame(all_vs,columns=final_genres_list+final_themes_list+final_features_list)
#print(len(v_df))
#print(len(appwarnings_df))
v_df['appid']=appwarnings_df['appid']
v_df=v_df.set_index('appid')
v_df.to_csv(os.path.join(parent_folder,"bulk","feature_vecs",'nonnormalized.csv'),index=True)
v_df.head()

,2D,4X,Action,Adventure,Arcade,Battle Royale,Beat 'em up,Board Game,Building,Card Game,...,Multiplayer,Physics,Pixel Graphics,Procedural Generation,Resource Management,Rogue-lite,Split Screen,Story Rich,Team-Based,VR
appid,,,,,,,,,,,,,,,,,,,,,
15540,0.0,0.0,57.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0
16900,0.0,0.0,31.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,31.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31.0
25000,0.0,0.0,294.0,62.0,0.0,0.0,0.0,0.0,0.0,0.0,...,100.0,120.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
108600,247.0,0.0,0.0,209.0,0.0,0.0,0.0,0.0,408.0,0.0,...,1262.0,0.0,0.0,0.0,0.0,0.0,1262.0,0.0,0.0,0.0
116100,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,...,52.0,0.0,0.0,0.0,0.0,0.0,52.0,0.0,0.0,0.0


In [214]:
norm_df=v_df.apply(lambda x: x/np.linalg.norm(x),axis=1)
norm_df.head()
norm_df.to_csv(os.path.join(parent_folder,"bulk","feature_vecs",'fullnormalized.csv'),index=True)

In [215]:
test_v_df=pd.DataFrame.copy(v_df)

def partial_norm(x,idx_start,idx_end):
    if np.linalg.norm(x[idx_start:idx_end])!=0:
        x[idx_start:idx_end]=x[idx_start:idx_end]/np.linalg.norm(x[idx_start:idx_end])
    return x

cat_norm_df=test_v_df.apply(lambda x: partial_norm(x,0,len(final_genres_list)),axis=1)
cat_norm_df=cat_norm_df.apply(lambda x: partial_norm(x,len(final_genres_list),len(final_genres_list)+len(final_themes_list)),axis=1)
cat_norm_df=cat_norm_df.apply(lambda x: partial_norm(x,len(final_genres_list)+len(final_themes_list),len(final_genres_list)+len(final_themes_list)+len(final_features_list)),axis=1)

cat_norm_df.to_csv(os.path.join(parent_folder,"bulk","feature_vecs",'catnormalized.csv'),index=True)

In [200]:
cat_norm_df.head()

,2D,4X,Action,Adventure,Arcade,Battle Royale,Beat 'em up,Board Game,Building,Card Game,...,Multiplayer,Physics,Pixel Graphics,Procedural Generation,Resource Management,Rogue-lite,Split Screen,Story Rich,Team-Based,VR
appid,,,,,,,,,,,,,,,,,,,,,
15540,0.000000,0.0,0.675373,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.00000,0.000000,0.0,1.0,0.0,0.0,0.00000,0.0,0.0,0.0
16900,0.000000,0.0,0.541779,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.50000,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.5
25000,0.000000,0.0,0.561443,0.118400,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.20772,0.249264,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0
108600,0.086515,0.0,0.000000,0.073205,0.000000,0.0,0.0,0.0,0.142908,0.0,...,0.49069,0.000000,0.0,0.0,0.0,0.0,0.49069,0.0,0.0,0.0
116100,0.000000,0.0,0.000000,0.000000,0.161048,0.0,0.0,0.0,0.000000,0.0,...,0.50000,0.000000,0.0,0.0,0.0,0.0,0.50000,0.0,0.0,0.0


In [217]:
dot_df=cat_norm_df.dot(norm_df.loc[244850])
dot_df=dot_df.sort_values(ascending = False)
print(dot_df.head(n=10))

appid
244850    1.657509
544550    1.454520
363360    1.450850
244770    1.434263
383120    1.433140
445220    1.400713
271240    1.368896
282860    1.346607
329130    1.335847
252870    1.313734
dtype: float64


In [137]:
final_genres_list=['2D','4X','Survival','Open World','Adventure','Action','RPG','Indie','Dungeon Crawler','Casual','Strategy',
        'Simulation','Sandbox','Shooter','Horror','Sports','Visual Novel','First-Person',
        'Top-Down','Third Person','God Game','Rhythm',"Beat 'em up",'Survival Horror','Battle Royale',
        'Puzzle','PvP','Rogue-like','Tactics','Turn-Based','Real-Time','MOBA','Building','VR Only',
        'Platformer','Walking Simulator','Stealth','Side Scroller','Education','Metroidvania','Space Sim',
        'Management','2D Fighter','Fighting','Hack and Slash','Flight','Exploration','Isometric','Runner',
        "Shoot 'Em Up",'Arcade','Racing','Board Game','Card Game','Software']
final_themes_list=['Western','Sci-fi','Anime','Realistic','Cyberpunk', 'Historical','Cute', 'Lovecraftian',
                   'Space','Relaxing', 'Magic','Retro','Atmospheric','Fantasy','Science','Steampunk',
                   'Zombies','Futuristic','Minimalist','Funny', 'Post-apocalyptic','Dinosaurs','Medieval',
                   'Tactical','Robots','Military','Naval','Colorful','War','Music']
final_features_list=['Pixel Gra

4925


In [157]:
print(all_vs[0]/np.linalg.norm(all_vs[0]))

[0.         0.         0.         0.         0.         0.56852375
 0.         0.56852375 0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.24935252 0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.47875685 0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.24935252 0.
 0.         0.         0.         0.         0. 

In [181]:
print('<option selected="selected" value = "-1">Select genre:</option>')
for i in range(len(final_genres_list)):
    print('<option value="'+str(i)+'">'+final_genres_list[i]+'</option>')

<option selected="selected" value = "-1">Select genre:</option>
<option value="0">2D</option>
<option value="1">4X</option>
<option value="2">Action</option>
<option value="3">Adventure</option>
<option value="4">Arcade</option>
<option value="5">Battle Royale</option>
<option value="6">Beat 'em up</option>
<option value="7">Board Game</option>
<option value="8">Building</option>
<option value="9">Card Game</option>
<option value="10">Casual</option>
<option value="11">Dungeon Crawler</option>
<option value="12">Education</option>
<option value="13">Exploration</option>
<option value="14">Fighting</option>
<option value="15">First-Person</option>
<option value="16">Flight</option>
<option value="17">God Game</option>
<option value="18">Hack and Slash</option>
<option value="19">Horror</option>
<option value="20">Indie</option>
<option value="21">Isometric</option>
<option value="22">MOBA</option>
<option value="23">Management</option>
<option value="24">Metroidvania</option>
<option va

In [183]:
print('<option selected="selected" value = "-1">Select theme:</option>')
for i in range(len(final_themes_list)):
    print('<option value="'+str(i+len(final_genres_list))+'">'+final_themes_list[i]+'</option>')

<option selected="selected" value = "-1">Select theme:</option>
<option value="54">Anime</option>
<option value="55">Atmospheric</option>
<option value="56">Colorful</option>
<option value="57">Cute</option>
<option value="58">Cyberpunk</option>
<option value="59">Dinosaurs</option>
<option value="60">Fantasy</option>
<option value="61">Funny</option>
<option value="62">Futuristic</option>
<option value="63">Historical</option>
<option value="64">Lovecraftian</option>
<option value="65">Magic</option>
<option value="66">Medieval</option>
<option value="67">Military</option>
<option value="68">Minimalist</option>
<option value="69">Music</option>
<option value="70">Naval</option>
<option value="71">Post-apocalyptic</option>
<option value="72">Realistic</option>
<option value="73">Relaxing</option>
<option value="74">Retro</option>
<option value="75">Robots</option>
<option value="76">Sci-fi</option>
<option value="77">Science</option>
<option value="78">Space</option>
<option value="79"

In [197]:
print('<option selected="selected" value = "-1">Select feature:</option>')
for i in range(len(final_features_list)):
    print('<option value="'+str(i+len(final_genres_list)+len(final_themes_list))+'">'+final_features_list[i]+'</option>')

<option selected="selected" value = "-1">Select feature:</option>
<option value="84">Character Customization</option>
<option value="85">Co-op</option>
<option value="86">Competitive</option>
<option value="87">Controller</option>
<option value="88">Crafting</option>
<option value="89">Difficult</option>
<option value="90">Downloadable Content</option>
<option value="91">Fast-Paced</option>
<option value="92">In-App Purchases</option>
<option value="93">Level Editor</option>
<option value="94">Massively Multiplayer</option>
<option value="95">Multiplayer</option>
<option value="96">Physics</option>
<option value="97">Pixel Graphics</option>
<option value="98">Procedural Generation</option>
<option value="99">Resource Management</option>
<option value="100">Rogue-lite</option>
<option value="101">Split Screen</option>
<option value="102">Story Rich</option>
<option value="103">Team-Based</option>
<option value="104">VR</option>


In [196]:
print(len(final_genres_list)+len(final_themes_list)+len(final_features_list))

105


In [216]:
print(len(final_genres_list))

54
